In [1]:
import random

random.seed(14)

In [2]:
import time

NOTEBOOK_START_TIME = time.time()

In [3]:
import gc
import math
import re
import sys
import subprocess
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from collections import Counter
from numpy.random import choice
PRIVATE = True

In [ ]:
#!pip install -U transformers --upgrade

In [56]:
import torch
import gc
torch.backends.cuda.enable_mem_efficient_sdp(False)

from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    AutoConfig,
    StoppingCriteria,
    StoppingCriteriaList,
    set_seed
)

import transformers
print(f"Transformers Version: {transformers.__version__}")
set_seed(14)

Transformers Version: 4.41.2


In [5]:
MODEL_PATH = "/kaggle/input/gemma/transformers/2b-it/3"#"/kaggle/input/gemma/transformers/7b-it/1"
DEEP = True

config = AutoConfig.from_pretrained(MODEL_PATH)
config.gradient_checkpointing = True

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
    
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    device_map='auto',
    torch_dtype="auto",
    trust_remote_code=True,
    config=config
    )

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype='auto',
    device_map='auto',
    )

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
gc.collect()

21

In [7]:
class StoppingCriteriaSub(StoppingCriteria):
    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop.to("cuda") for stop in stops]
        
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            last_token = input_ids[0][-len(stop):]
            if torch.all(torch.eq(stop,last_token)):
                return True
        return False


stop_words = ["```output", "```python", "```\nOutput" , ")\n```" , "``````output"] #,  
stop_words_ids = [tokenizer(stop_word, return_tensors='pt', add_special_tokens=False)['input_ids'].squeeze() for stop_word in stop_words]
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])

In [28]:
USE_PAST_KEY = True

temperature = 0.90
top_p = 1.0

temperature_coding = 0.90
top_p_coding = 1.0


In [9]:
fd=pd.read_csv('/kaggle/input/prodata/probability.csv')

In [10]:
fd.head()

,problem,answer
0,Alice has 2 kids and one of them is a girl. Wh...,3.330000e-01
1,A fair six-sided die is rolled twice. What is ...,1.388889e-01
2,"Consider a tetrahedral , A tetrahedral die has...",3.750000e-01
3,Anita randomly picks 4 cards from a deck of 52...,5.271878e+10
4,"If you dealt 13 cards, what is the probability...",7.692308e-02


In [20]:
pro = """Below is a stat problem you are to consider:
\"{}\"
To solve the problem, think step by step and reach the solution. Support your answer by python code. The result may be real number
Approach:"""

In [43]:
from IPython.display import Markdown as md

In [54]:
k=len(fd['problem'].loc[:])
sol=[]
for i in range (0,k):
    problem=fd['problem'].loc[i]
    initail_message = pro.format(problem,"{}")
    prompt = f"User: {initail_message}"
    print(prompt)
    model_inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    input_len = len(model_inputs['input_ids'][0])
    generation_output = model.generate(**model_inputs,
                                       max_new_tokens=2048,
                                       return_dict_in_generate=USE_PAST_KEY,
                                       do_sample = True,
                                       temperature = temperature,
                                       top_p = top_p,
                                       num_return_sequences=6)##, stopping_criteria = stopping_criteria
    output_ids = generation_output.sequences[0]
    decoded_output = tokenizer.decode(output_ids, skip_special_tokens=True)
    print(decoded_output)

User: Below is a stat problem you are to consider:
"Alice has 2 kids and one of them is a girl. What is the probability that the other child is also a girl?"
To solve the problem, think step by step and reach the solution. Support your answer by python code. The result may be real number
Approach:
User: Below is a stat problem you are to consider:
"Alice has 2 kids and one of them is a girl. What is the probability that the other child is also a girl?"
To solve the problem, think step by step and reach the solution. Support your answer by python code. The result may be real number
Approach:

**Step 1**: Define the variables

* a: the number of children Alice has
* b: the number of girls in Alice's children

**Step 2**: Write a formula to calculate the probability

**Step 3**: Substitute the values of a and b into the formula

**Step 4**: Calculate the probability

**Step 5**: Print the probability

```python
# Define the variables
a = 2
b = 1

# Calculate the probability
probability = 